In [101]:
import json
import sys
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize.stanford import StanfordTokenizer
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import re
import string
import nltk
import numpy as np
import pandas as pd
#nltk.download() #use if nltk cries for any language-specific docs

In [2]:
"""ay=str()
for file in os.listdir('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10'):
    full_filename = "%s/%s" % ('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10', file)
    if full_filename.endswith('.txt'):
        with open(full_filename,'r', encoding="ISO-8859-1") as fi:
            ay=fi.readlines()
"""
subs=[]
for filename in os.listdir('Subs'):
    if filename.endswith(".json"):
        with open(os.path.join('Subs', filename)) as json_file:
            json_text = json.load(json_file)
            print(filename)
            subs.append(json_text)

OUTPUT200acres.json
OUTPUTBreadTube.json
OUTPUTEarthStrike.json
OUTPUTPhilosophyTube.json


In [3]:
#not used
def sub2vocab(sub):
    documents=sub2docs(sub)
    temp=[]
    stop = set(stopwords.words('english'))
    #for doc in documents:
        #print(word_tokenize(doc))
    texts =[
        [word for word in document.lower().split() if word not in stop]
        for document in documents
    ]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts]
    return texts

In [4]:
#not used even more
"""def sub2vocab(sub):
    raw=str()
    for i, j in sub.items():
        raw+=i
        raw+=' '
        for o,p in j.items():
            raw+=p
            raw+=' '
        raw+='\n'
    temp=[]
    stop = set(stopwords.words('english'))
    for i in word_tokenize(raw.lower()):
        if i not in stop:
            temp.append(i)
    frequency = defaultdict(int)
    for word in temp:
        frequency[word] += 1
    temp = set([token for token in temp if frequency[token] > 1])
    return temp"""

In [65]:
def sub2docs(sub):
    documents=[]
    trans_table = str.maketrans("", "", string.punctuation)
    for i, j in sub.items():
        temp=re.sub(r'\d+','',i.lower().translate(trans_table)).replace('\n', '')
        documents.append(temp)
        for o,p in j.items():
            temp=re.sub(r'\d+', '',p.lower().translate(trans_table)).replace('\n', '')
            documents.append(temp)
    frequency = defaultdict(int)
    for document in documents:
        for token in document.split():
            frequency[token] += 1
    documents = [
        [token for token in document.split() if frequency[token] > 1]
        for document in documents]
    return documents

In [ ]:
def removePunctuation(words):
    trans_table = str.maketrans("", "", string.punctuation)
    return set([s.translate(trans_table) for s in words])

In [67]:
def target_wv2distances(wv):
    dists=np.zeros((len(wv.vocab),len(wv.vocab)))
    words=[]
    for word in wv.vocab.keys():
        words.append(word)
    start=0
    for i in range(len(keys)):
        print(i,len(keys))
        for j in range(len(keys)):
            j+=start
            if j<len(keys):
                dists[i,j]=wv.n_similarity([words[i]],[words[j]])
        start+=1
    return dists

In [79]:
def ref_wv2distances(target_wv,reference_wv):
    dists=np.zeros((len(target_wv.vocab),len(target_wv.vocab)))
    words=[]
    for word in target_wv.vocab.keys():
        words.append(word)
    start=0
    for i in range(len(keys)):
        print(i,len(keys))
        for j in range(len(keys)):
            j+=start
            try:
                if j<len(keys):
                    dists[i,j]=word_vectors.n_similarity([words[i]],[words[j]])
            except Exception as e:
                pass
                #print("type error: " + str(e))
        start+=1
    return dists


In [32]:
def docs2wv(docs,name='model'):
    model = Word2Vec(docs, size=300, window=5, workers=3, sg=1, min_count=1, negative=8)
    if name!=None:
        model.save(name+".model")
    word_vectors = model.wv
    return word_vectors

In [97]:
def remove_diagonal(dists):
    for i in range(dists.shape[0]):
        for j in range(dists.shape[1]):
            if i==j:
                dists[i,j]=0
    return dists

In [70]:
target_wv=docs2wv(sub2docs(subs[0]),name='target')

E:\Anaconda33\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [73]:
ref_wv=gensim.models.KeyedVectors.load_word2vec_format(os.path.join(os.path.dirname('Models\word2vec\English_CoNLL17\ '), 'model.bin'), binary=True).wv

E:\Anaconda33\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [80]:
dists_ref=ref_wv2distances(target_wv,ref_wv)
dists_t=target_wv2distances(target_wv)

0 6651
1 6651
2 6651
3 6651
4 6651
5 6651
6 6651
7 6651
8 6651
9 6651
10 6651
11 6651
12 6651
13 6651
14 6651
15 6651
16 6651
17 6651
18 6651
19 6651
20 6651
21 6651
22 6651
23 6651
24 6651
25 6651
26 6651
27 6651
28 6651
29 6651
30 6651
31 6651
32 6651
33 6651
34 6651
35 6651
36 6651
37 6651
38 6651
39 6651
40 6651
41 6651
42 6651
43 6651
44 6651
45 6651
46 6651
47 6651
48 6651
49 6651
50 6651
51 6651
52 6651
53 6651
54 6651
55 6651
56 6651
57 6651
58 6651
59 6651
60 6651
61 6651
62 6651
63 6651
64 6651
65 6651
66 6651
67 6651
68 6651
69 6651
70 6651
71 6651
72 6651
73 6651
74 6651
75 6651
76 6651
77 6651
78 6651
79 6651
80 6651
81 6651
82 6651
83 6651
84 6651
85 6651
86 6651
87 6651
88 6651
89 6651
90 6651
91 6651
92 6651
93 6651
94 6651
95 6651
96 6651
97 6651
98 6651
99 6651
100 6651
101 6651
102 6651
103 6651
104 6651
105 6651
106 6651
107 6651
108 6651
109 6651
110 6651
111 6651
112 6651
113 6651
114 6651
115 6651
116 6651
117 6651
118 6651
119 6651
120 6651
121 6651
122 6651
123

923 6651
924 6651
925 6651
926 6651
927 6651
928 6651
929 6651
930 6651
931 6651
932 6651
933 6651
934 6651
935 6651
936 6651
937 6651
938 6651
939 6651
940 6651
941 6651
942 6651
943 6651
944 6651
945 6651
946 6651
947 6651
948 6651
949 6651
950 6651
951 6651
952 6651
953 6651
954 6651
955 6651
956 6651
957 6651
958 6651
959 6651
960 6651
961 6651
962 6651
963 6651
964 6651
965 6651
966 6651
967 6651
968 6651
969 6651
970 6651
971 6651
972 6651
973 6651
974 6651
975 6651
976 6651
977 6651
978 6651
979 6651
980 6651
981 6651
982 6651
983 6651
984 6651
985 6651
986 6651
987 6651
988 6651
989 6651
990 6651
991 6651
992 6651
993 6651
994 6651
995 6651
996 6651
997 6651
998 6651
999 6651
1000 6651
1001 6651
1002 6651
1003 6651
1004 6651
1005 6651
1006 6651
1007 6651
1008 6651
1009 6651
1010 6651
1011 6651
1012 6651
1013 6651
1014 6651
1015 6651
1016 6651
1017 6651
1018 6651
1019 6651
1020 6651
1021 6651
1022 6651
1023 6651
1024 6651
1025 6651
1026 6651
1027 6651
1028 6651
1029 6651
1030 66

1751 6651
1752 6651
1753 6651
1754 6651
1755 6651
1756 6651
1757 6651
1758 6651
1759 6651
1760 6651
1761 6651
1762 6651
1763 6651
1764 6651
1765 6651
1766 6651
1767 6651
1768 6651
1769 6651
1770 6651
1771 6651
1772 6651
1773 6651
1774 6651
1775 6651
1776 6651
1777 6651
1778 6651
1779 6651
1780 6651
1781 6651
1782 6651
1783 6651
1784 6651
1785 6651
1786 6651
1787 6651
1788 6651
1789 6651
1790 6651
1791 6651
1792 6651
1793 6651
1794 6651
1795 6651
1796 6651
1797 6651
1798 6651
1799 6651
1800 6651
1801 6651
1802 6651
1803 6651
1804 6651
1805 6651
1806 6651
1807 6651
1808 6651
1809 6651
1810 6651
1811 6651
1812 6651
1813 6651
1814 6651
1815 6651
1816 6651
1817 6651
1818 6651
1819 6651
1820 6651
1821 6651
1822 6651
1823 6651
1824 6651
1825 6651
1826 6651
1827 6651
1828 6651
1829 6651
1830 6651
1831 6651
1832 6651
1833 6651
1834 6651
1835 6651
1836 6651
1837 6651
1838 6651
1839 6651
1840 6651
1841 6651
1842 6651
1843 6651
1844 6651
1845 6651
1846 6651
1847 6651
1848 6651
1849 6651
1850 6651


2572 6651
2573 6651
2574 6651
2575 6651
2576 6651
2577 6651
2578 6651
2579 6651
2580 6651
2581 6651
2582 6651
2583 6651
2584 6651
2585 6651
2586 6651
2587 6651
2588 6651
2589 6651
2590 6651
2591 6651
2592 6651
2593 6651
2594 6651
2595 6651
2596 6651
2597 6651
2598 6651
2599 6651
2600 6651
2601 6651
2602 6651
2603 6651
2604 6651
2605 6651
2606 6651
2607 6651
2608 6651
2609 6651
2610 6651
2611 6651
2612 6651
2613 6651
2614 6651
2615 6651
2616 6651
2617 6651
2618 6651
2619 6651
2620 6651
2621 6651
2622 6651
2623 6651
2624 6651
2625 6651
2626 6651
2627 6651
2628 6651
2629 6651
2630 6651
2631 6651
2632 6651
2633 6651
2634 6651
2635 6651
2636 6651
2637 6651
2638 6651
2639 6651
2640 6651
2641 6651
2642 6651
2643 6651
2644 6651
2645 6651
2646 6651
2647 6651
2648 6651
2649 6651
2650 6651
2651 6651
2652 6651
2653 6651
2654 6651
2655 6651
2656 6651
2657 6651
2658 6651
2659 6651
2660 6651
2661 6651
2662 6651
2663 6651
2664 6651
2665 6651
2666 6651
2667 6651
2668 6651
2669 6651
2670 6651
2671 6651


3393 6651
3394 6651
3395 6651
3396 6651
3397 6651
3398 6651
3399 6651
3400 6651
3401 6651
3402 6651
3403 6651
3404 6651
3405 6651
3406 6651
3407 6651
3408 6651
3409 6651
3410 6651
3411 6651
3412 6651
3413 6651
3414 6651
3415 6651
3416 6651
3417 6651
3418 6651
3419 6651
3420 6651
3421 6651
3422 6651
3423 6651
3424 6651
3425 6651
3426 6651
3427 6651
3428 6651
3429 6651
3430 6651
3431 6651
3432 6651
3433 6651
3434 6651
3435 6651
3436 6651
3437 6651
3438 6651
3439 6651
3440 6651
3441 6651
3442 6651
3443 6651
3444 6651
3445 6651
3446 6651
3447 6651
3448 6651
3449 6651
3450 6651
3451 6651
3452 6651
3453 6651
3454 6651
3455 6651
3456 6651
3457 6651
3458 6651
3459 6651
3460 6651
3461 6651
3462 6651
3463 6651
3464 6651
3465 6651
3466 6651
3467 6651
3468 6651
3469 6651
3470 6651
3471 6651
3472 6651
3473 6651
3474 6651
3475 6651
3476 6651
3477 6651
3478 6651
3479 6651
3480 6651
3481 6651
3482 6651
3483 6651
3484 6651
3485 6651
3486 6651
3487 6651
3488 6651
3489 6651
3490 6651
3491 6651
3492 6651


...

2602 6651
2603 6651
2604 6651
2605 6651
2606 6651
2607 6651
2608 6651
2609 6651
2610 6651
2611 6651
2612 6651
2613 6651
2614 6651
2615 6651
2616 6651
2617 6651
2618 6651
2619 6651
2620 6651
2621 6651
2622 6651
2623 6651
2624 6651
2625 6651
2626 6651
2627 6651
2628 6651
2629 6651
2630 6651
2631 6651
2632 6651
2633 6651
2634 6651
2635 6651
2636 6651
2637 6651
2638 6651
2639 6651
2640 6651
2641 6651
2642 6651
2643 6651
2644 6651
2645 6651
2646 6651
2647 6651
2648 6651
2649 6651
2650 6651
2651 6651
2652 6651
2653 6651
2654 6651
2655 6651
2656 6651
2657 6651
2658 6651
2659 6651
2660 6651
2661 6651
2662 6651
2663 6651
2664 6651
2665 6651
2666 6651
2667 6651
2668 6651
2669 6651
2670 6651
2671 6651
2672 6651
2673 6651
2674 6651
2675 6651
2676 6651
2677 6651
2678 6651
2679 6651
2680 6651
2681 6651
2682 6651
2683 6651
2684 6651
2685 6651
2686 6651
2687 6651
2688 6651
2689 6651
2690 6651
2691 6651
2692 6651
2693 6651
2694 6651
2695 6651
2696 6651
2697 6651
2698 6651
2699 6651
2700 6651
2701 6651


3422 6651
3423 6651
3424 6651
3425 6651
3426 6651
3427 6651
3428 6651
3429 6651
3430 6651
3431 6651
3432 6651
3433 6651
3434 6651
3435 6651
3436 6651
3437 6651
3438 6651
3439 6651
3440 6651
3441 6651
3442 6651
3443 6651
3444 6651
3445 6651
3446 6651
3447 6651
3448 6651
3449 6651
3450 6651
3451 6651
3452 6651
3453 6651
3454 6651
3455 6651
3456 6651
3457 6651
3458 6651
3459 6651
3460 6651
3461 6651
3462 6651
3463 6651
3464 6651
3465 6651
3466 6651
3467 6651
3468 6651
3469 6651
3470 6651
3471 6651
3472 6651
3473 6651
3474 6651
3475 6651
3476 6651
3477 6651
3478 6651
3479 6651
3480 6651
3481 6651
3482 6651
3483 6651
3484 6651
3485 6651
3486 6651
3487 6651
3488 6651
3489 6651
3490 6651
3491 6651
3492 6651
3493 6651
3494 6651
3495 6651
3496 6651
3497 6651
3498 6651
3499 6651
3500 6651
3501 6651
3502 6651
3503 6651
3504 6651
3505 6651
3506 6651
3507 6651
3508 6651
3509 6651
3510 6651
3511 6651
3512 6651
3513 6651
3514 6651
3515 6651
3516 6651
3517 6651
3518 6651
3519 6651
3520 6651
3521 6651


4244 6651
4245 6651
4246 6651
4247 6651
4248 6651
4249 6651
4250 6651
4251 6651
4252 6651
4253 6651
4254 6651
4255 6651
4256 6651
4257 6651
4258 6651
4259 6651
4260 6651
4261 6651
4262 6651
4263 6651
4264 6651
4265 6651
4266 6651
4267 6651
4268 6651
4269 6651
4270 6651
4271 6651
4272 6651
4273 6651
4274 6651
4275 6651
4276 6651
4277 6651
4278 6651
4279 6651
4280 6651
4281 6651
4282 6651
4283 6651
4284 6651
4285 6651
4286 6651
4287 6651
4288 6651
4289 6651
4290 6651
4291 6651
4292 6651
4293 6651
4294 6651
4295 6651
4296 6651
4297 6651
4298 6651
4299 6651
4300 6651
4301 6651
4302 6651
4303 6651
4304 6651
4305 6651
4306 6651
4307 6651
4308 6651
4309 6651
4310 6651
4311 6651
4312 6651
4313 6651
4314 6651
4315 6651
4316 6651
4317 6651
4318 6651
4319 6651
4320 6651
4321 6651
4322 6651
4323 6651
4324 6651
4325 6651
4326 6651
4327 6651
4328 6651
4329 6651
4330 6651
4331 6651
4332 6651
4333 6651
4334 6651
4335 6651
4336 6651
4337 6651
4338 6651
4339 6651
4340 6651
4341 6651
4342 6651
4343 6651


5064 6651
5065 6651
5066 6651
5067 6651
5068 6651
5069 6651
5070 6651
5071 6651
5072 6651
5073 6651
5074 6651
5075 6651
5076 6651
5077 6651
5078 6651
5079 6651
5080 6651
5081 6651
5082 6651
5083 6651
5084 6651
5085 6651
5086 6651
5087 6651
5088 6651
5089 6651
5090 6651
5091 6651
5092 6651
5093 6651
5094 6651
5095 6651
5096 6651
5097 6651
5098 6651
5099 6651
5100 6651
5101 6651
5102 6651
5103 6651
5104 6651
5105 6651
5106 6651
5107 6651
5108 6651
5109 6651
5110 6651
5111 6651
5112 6651
5113 6651
5114 6651
5115 6651
5116 6651
5117 6651
5118 6651
5119 6651
5120 6651
5121 6651
5122 6651
5123 6651
5124 6651
5125 6651
5126 6651
5127 6651
5128 6651
5129 6651
5130 6651
5131 6651
5132 6651
5133 6651
5134 6651
5135 6651
5136 6651
5137 6651
5138 6651
5139 6651
5140 6651
5141 6651
5142 6651
5143 6651
5144 6651
5145 6651
5146 6651
5147 6651
5148 6651
5149 6651
5150 6651
5151 6651
5152 6651
5153 6651
5154 6651
5155 6651
5156 6651
5157 6651
5158 6651
5159 6651
5160 6651
5161 6651
5162 6651
5163 6651


5884 6651
5885 6651
5886 6651
5887 6651
5888 6651
5889 6651
5890 6651
5891 6651
5892 6651
5893 6651
5894 6651
5895 6651
5896 6651
5897 6651
5898 6651
5899 6651
5900 6651
5901 6651
5902 6651
5903 6651
5904 6651
5905 6651
5906 6651
5907 6651
5908 6651
5909 6651
5910 6651
5911 6651
5912 6651
5913 6651
5914 6651
5915 6651
5916 6651
5917 6651
5918 6651
5919 6651
5920 6651
5921 6651
5922 6651
5923 6651
5924 6651
5925 6651
5926 6651
5927 6651
5928 6651
5929 6651
5930 6651
5931 6651
5932 6651
5933 6651
5934 6651
5935 6651
5936 6651
5937 6651
5938 6651
5939 6651
5940 6651
5941 6651
5942 6651
5943 6651
5944 6651
5945 6651
5946 6651
5947 6651
5948 6651
5949 6651
5950 6651
5951 6651
5952 6651
5953 6651
5954 6651
5955 6651
5956 6651
5957 6651
5958 6651
5959 6651
5960 6651
5961 6651
5962 6651
5963 6651
5964 6651
5965 6651
5966 6651
5967 6651
5968 6651
5969 6651
5970 6651
5971 6651
5972 6651
5973 6651
5974 6651
5975 6651
5976 6651
5977 6651
5978 6651
5979 6651
5980 6651
5981 6651
5982 6651
5983 6651


In [98]:
dists_ref = remove_diagonal(dists_ref)

In [99]:
dists_t  = remove_diagonal(dists_t)

In [105]:
ranks=pd.DataFrame(columns=['Value'])
i=0
dists_comp=dists_t-dists_ref
for word in target_wv.vocab.keys():
    temp=np.sum(dists_comp[i,:])+np.sum(dists_comp[:,i])
    ranks.loc[word]=temp
    i+=1

In [110]:
ranks.sort_values(['Value'])

,Value
thomas,-2712.000874
c,-2699.775692
johnson,-2529.949951
martin,-2486.582962
albert,-2411.148520
george,-2354.922624
francis,-2338.555562
frederic,-2331.088816
jasper,-2318.041177
bierstadt,-2202.896176
